In [ ]:
import os
import numpy as np
import torch
import torch.nn as nn
import torchvision.models as models
from torch.autograd import Variable
import skvideo.io
import time
import math

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

batch_size = 1  # Extract Resnet features in a tensor of batch_size videos.
total_ext = 1000  #  Extract Resnet features of total_ext videos

resnet_feat_path = '../dataset_224/activitynet/res_feature.pt'
batch_ckpt_path = '../dataset_224/activitynet/batch_num.txt'
video_path = '../dataset_224/activitynet/videos/'

# Download pre trained Resnet101.
resnet50 = models.resnet50(pretrained=True)
modules = list(resnet50.children())[:-1]
resnet50 = nn.Sequential(*modules)
for p in resnet50.parameters():
    p.requires_grad = False
# resnet50 = resnet50.to(device)

In [ ]:
# Convert id's list to dictionary with key as id, value as id numpy array
def video_array(batch_ids):
    batch_array = {}
    for ids in batch_ids:
        video = skvideo.io.vread(video_path + ids)
        batch_array.update({ids:video})
    return batch_array

In [ ]:
# Convert array dictionary to resnet_dictionary with key as id, value as id tensor feature array
def resnet_features(batch_arrayd):
    batch_feature = {}
    ids = list(batch_arrayd.keys())
    video_array = [x for x in batch_arrayd.values()]
    array_sizes = [x.shape[0] for x in batch_arrayd.values()]

    video1_array = np.array(video_array[0], dtype = np.float32)  # change datatype of frames to float32
    video_tensor = torch.from_numpy(video1_array)

    if batch_size > 1:
        for i in range(1, len(video_array)):
            videoi_array = np.array(video_array[i], dtype = np.float32)
            videoi_tensor = torch.from_numpy(videoi_array)
            video_tensor = torch.cat((video_tensor, videoi_tensor), 0)

    video_tensor = video_tensor
    video_tensor = video_tensor.permute(0,3,1,2) # change dimension to [?,3,224,224]
    print(type(video_tensor))
    tensor_var = Variable(video_tensor)
    resnet_feature = resnet50(tensor_var).data
    resnet_feature.squeeze_(3)  # eliminate last dimension to get [?, 2048, 1, 1]
    resnet_feature.squeeze_(2)  # eliminate last dimension to get [?, 2048]

    index = 0
    for i in range(len(video_array)):
        id_tensor = resnet_feature[index:index+array_sizes[i]]
        batch_feature.update({ids[i]:id_tensor})
        index += array_sizes[i]

    return batch_feature

In [ ]:
a = os.listdir(video_path)
print(len(a))
print(a[0])

In [ ]:
b = torch.FloatTensor([1,2])
print(b)

In [ ]:
# seconds to minutes
def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return m, s

# time in minutes b/w since to now
def timeSince(since):
    now = time.time()
    s = now - since
    min, sec = asMinutes(s) 
    return min, sec


if __name__ == '__main__':

    print('Total videos', total_ext)
    print('batch size', batch_size)
    total_iter = math.ceil(total_ext / batch_size)

    featured = {}
    
    all_ids = os.listdir(video_path)[:total_ext]

    if os.path.exists(batch_ckpt_path):
        with open(batch_ckpt_path,'r') as f:
            content = f.readline().rstrip()
            batch_start = int(content)
    else:
        batch_start =  0

    batch_end = min(batch_start+batch_size,total_ext) 

    iter = 1
    while batch_start != batch_end:

        print('Iteration', iter, 'left', total_iter-iter, end=' ')
        start_time = time.time()
        batch_id = all_ids[batch_start:batch_start+batch_size]
        batch_arrayd = video_array(batch_id)
        batch_featuresd = resnet_features(batch_arrayd) 
        featured.update(batch_featuresd)
        print('time taken (%dm %ds)'% timeSince(start_time))

        # Save resnet features to pytorch file
        state = featured
        torch.save(state, resnet_feat_path)  

        # save batch_start to batch_ckpt_path file
        with open(batch_ckpt_path, 'w') as f:
            f.write(str(batch_end))

        batch_start = batch_end 
        batch_end = min(batch_start+batch_size, total_ext)
        iter += 1